In [18]:
!pip install datasets tqdm pandas

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
!pip install sentencepiece

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [21]:
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
!pip install wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

In [24]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Sun Jan 21 19:15:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0              28W /  70W |   3321MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [25]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    AutoTokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [26]:
import random
import numpy as np
import torch
import datasets

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)

In [27]:
pd.set_option('display.max_colwidth', None)

In [28]:
df = pd.read_csv('/kaggle/input/80-20-rev/train8020.csv')
df.shape


(20084, 10)

In [29]:
from transformers import (
    T5ForConditionalGeneration, AutoTokenizer,
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
  )

from torch.utils.data import Dataset, DataLoader


In [31]:
def calc_token_len(example):
    return len(tokenizer(example).input_ids)


In [32]:
train_df = pd.read_csv('/kaggle/input/80-20-rev/train8020.csv')
test_df = pd.read_csv('/kaggle/input/80-20-rev/test8020.csv')

In [33]:
model_name = 'csebuetnlp/banglat5_small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [34]:
!pip install git+https://github.com/csebuetnlp/normalizer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-3oqnsrb8
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-3oqnsrb8
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 1.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6857 sha256=a396b05414f9eb65a246a1531b89f745a1413b07c8763bb062209bc0f4d03a9c
  Stored in directory: /tmp/pip-ephem-wheel-cache-7bt4opfa/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186459 sha256=aeb0edd8a36faec48630b6b2d9a7125

In [35]:
from normalizer import normalize
train_df['Comment'] = train_df['Comment'].apply(normalize)
train_df['Correct Form'] = train_df['Correct Form'].apply(normalize)
test_df['Comment'] = test_df['Comment'].apply(normalize)
test_df['Correct Form'] = test_df['Correct Form'].apply(normalize)

In [36]:
train_df.shape, test_df.shape

((20084, 10), (5022, 10))

In [37]:
def calc_token_len(example):
    return len(tokenizer(example).input_ids)

train_df['input_token_len'] = train_df['Comment'].apply(calc_token_len)
test_df['input_token_len'] = test_df['Comment'].apply(calc_token_len)
test_df.head()

,Video ID,Channel name,Time of Publishing,Title,Genre,Comment,Error,Category,Correct Form,Unnamed: 9,input_token_len
0,62d6PfZxky0,Bangla Bhuter Golpo,2023-01-16T13:00:48Z,Bhuter Cartoon - Railway Station at 2am Night (True Story) Train Horror Story | Bangla Bhuter Golpo,Entertainment,কথা হচ্ছে ওরা এতো ট্রেন পেল কোথায় 🙄 যে এক ঘন্টা পর পর ট্রেন আসতেছে 😳,1,Multiple Errors,কথা হচ্ছে ওরা এত ট্রেন পেল কোথায় 🙄 যে এক ঘণ্টা পর পর ট্রেন আসছে 😳,NaN,20
1,xgMi3vUaw5Y,Baseera,2022-04-02T19:03:14Z,সিরাহ ৭ – অ্যাবিসিনিয়া | Bangla Seerah,Miscellaneous,"প্রিয় ভাই আমার,, আপনি আর বেশি বেশি ভিডিও ছাড়েন,, যেন সুনে আমরা আরো বেশি উপকৃর্ত হতে পারি,,, অনেক অপেক্ষায় থাকি।। আল্লাহ জন্য ভালোবাসি আপনাকে।।",1,Spelling,"প্রিয় ভাই আমার,, আপনি আর বেশি বেশি ভিডিও ছাড়েন,, যেন শুনে আমরা আরো বেশি উপকৃত হতে পারি,,, অনেক অপেক্ষায় থাকি।। আল্লাহ জন্য ভালোবাসি আপনাকে।।",NaN,37
2,8Q7mPmy9eTI,BANGLADESHI REACTOR,2023-01-22T07:00:02Z,লাইভে খেজুর পাতার জা*ইঙ্গা বিক্রি করছে টনি আপা | Salman Muqtadir female version | দাদি দাদা কই,Entertainment,"আপু তোমাকে ধন্যবাদ এইভাবে সবাইকে রিপ্লাই দিয়ে শিক্ষা দেওয়ার জন্য,, তুমি যদি সুখে থাকতে পারো তাহলে ওদের চুলকায় কেনো।",1,Code Switching,"আপু তোমাকে ধন্যবাদ এইভাবে সবাইকে জবাব দিয়ে শিক্ষা দেওয়ার জন্য,, তুমি যদি সুখে থাকতে পারো তাহলে ওদের চুলকায় কেনো।",NaN,24
3,gMeeozaMmIM,Laser Vision,2018-01-13T11:04:17Z,Surja Dighal Bari | Bangla Movie | Dolly Anwar | Zahirul Haque | Rowshan Jamil | Sheikh Niamat Ali,Entertainment,"এখানে আমার মনে হয় কেউ অভিনয় করে নাই। এটা একদম জীবন এর সাথে মিলে গেছে। একদম নিখুঁত অভিনয়। আগের দিন এর ছবি, নাটক দেখতে সত্যি অনেক সুন্দর লাগে অভিনয় যেন নয় যেন আসল শুধু ক্যামেরা বন্দী করা হয়ছে জীবনটাকে 🙂🙂।খুব সুন্দর লাগল ছবি টা 🙂🙂",1,Spelling,"এখানে আমার মনে হয় কেউ অভিনয় করে নাই। এটা একদম জীবন এর সাথে মিলে গেছে। একদম নিখুঁত অভিনয়। আগের দিন এর ছবি, নাটক দেখতে সত্যি অনেক সুন্দর লাগে অভিনয় যেন নয় যেন আসল শুধু ক্যামেরা বন্দী করা হয়েছে জীবনটাকে 🙂🙂।খুব সুন্দর লাগল ছবি টা 🙂🙂",NaN,55
4,Vhj-_avEjs0,Ritu Hossain,2023-06-26T12:00:08Z,রিতু নিজের টাকায় রুম ডেকোরেশন করলো | My Home Tour VLOG | Ritu Hossain | Rakib Hossain,Entertainment,ওওআপি আমি তোমার রোম দেখার জন্য অনেক অনেক অনেক হেপি😊,1,Spelling,আপু আমি তোমার কক্ষ দেখার জন্য অনেক অনেক অনেক খুশি😊,NaN,16


In [38]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
test_dataset

Dataset({
    features: ['Video ID', 'Channel name ', 'Time of Publishing', 'Title', 'Genre', 'Comment', 'Error', 'Category', 'Correct Form', 'Unnamed: 9', 'input_token_len'],
    num_rows: 5022
})

In [39]:
from torch.utils.data import Dataset, DataLoader
class GrammarDataset(Dataset):
    def __init__(self, dataset, tokenizer,print_text=False):
        self.dataset = dataset
        self.pad_to_max_length = False
        self.tokenizer = tokenizer
        self.print_text = print_text
        self.max_len = 64

    def __len__(self):
        return len(self.dataset)


    def tokenize_data(self, example):
        input_, target_ = example['Comment'], example['Correct Form']

        # tokenize inputs
        tokenized_inputs = tokenizer(input_, pad_to_max_length=self.pad_to_max_length,
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        tokenized_targets = tokenizer(target_, pad_to_max_length=self.pad_to_max_length,
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        inputs={"input_ids": tokenized_inputs['input_ids'],
            "attention_mask": tokenized_inputs['attention_mask'],
            "labels": tokenized_targets['input_ids']
        }

        return inputs


    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset[index])

        if self.print_text:
            for k in inputs.keys():
                print(k, len(inputs[k]))

        return inputs


In [40]:
dataset = GrammarDataset(test_dataset, tokenizer, True)
print(dataset[121])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


input_ids 24
attention_mask 24
labels 24
{'input_ids': [290, 406, 79, 14, 541, 38, 4576, 2082, 1068, 5, 6673, 2008, 17, 30, 881, 840, 11777, 756, 376, 520, 3157, 20, 2, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [290, 406, 79, 14, 541, 38, 4576, 2082, 1068, 5, 158, 2630, 16332, 30, 881, 840, 11777, 756, 376, 520, 3157, 20, 2, 1]}


In [41]:
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=7693d834e0f0b84e21fa28a37db1ac2829cec1cdbab0cc93ea76bd1771f27437
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [42]:
from datasets import load_metric
rouge_metric = load_metric("rouge")

In [44]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')

In [45]:
!pip install transformers[torch]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [46]:
!pip install accelerate -U

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/a6/b9/44623bdb05595481107153182e7f4b9f2ef9d3b674938ad13842054dcbd8/accelerate-0.26.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.4 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.25.0
    Uninstalling accelerate-0.25.0:
      Successfully uninstalled accelerate-0.25.0


In [47]:
# defining training related arguments
batch_size = 16
args = Seq2SeqTrainingArguments(output_dir="weights",
                        evaluation_strategy="steps",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=2e-5,
                        num_train_epochs=100,
                        weight_decay=0.01,
                        save_total_limit=2,
                        predict_with_generate=True,
                        fp16 = True,
                        gradient_accumulation_steps = 6,
                        eval_steps = 500,
                        save_steps = 500,
                        load_best_model_at_end=True,
                        logging_dir="/logs",
                        report_to="wandb")


In [48]:
import nltk
nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=False)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [50]:
# defining trainer using 🤗
trainer = Seq2SeqTrainer(model=model,
                args=args,
                train_dataset= GrammarDataset(train_dataset, tokenizer),
                eval_dataset=GrammarDataset(test_dataset, tokenizer),
                tokenizer=tokenizer,
                data_collator=data_collator,
                compute_metrics=compute_metrics)

In [51]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
500,2.328500,1.827368,2.011900,0.807900,2.006200,2.023700,10.906200
1000,1.746600,1.559524,2.360400,0.858500,2.356300,2.369200,11.781600
1500,1.473800,1.255186,3.068900,0.970900,3.060400,3.068600,14.314400
2000,0.643000,0.364188,3.997300,1.406000,3.988400,3.997600,14.128400
2500,0.450100,0.337451,4.042800,1.441700,4.038000,4.048500,14.103100
3000,0.413200,0.324088,4.062800,1.463900,4.058100,4.066800,14.094400
3500,0.389600,0.312370,4.021300,1.444000,4.015500,4.025700,14.079600
4000,0.373600,0.306396,4.026700,1.450000,4.021600,4.028600,14.074700
4500,0.361500,0.299929,4.041300,1.450000,4.033400,4.037300,14.065700
5000,0.351500,0.294832,4.048400,1.450000,4.042800,4.047400,14.062700


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars

TrainOutput(global_step=10400, training_loss=0.5763367168719952, metrics={'train_runtime': 13015.7689, 'train_samples_per_second': 154.305, 'train_steps_per_second': 0.799, 'total_flos': 2.577231451521024e+16, 'train_loss': 0.5763367168719952, 'epoch': 99.36})

In [54]:
trainer.save_model('bangla_gec_model')

In [53]:


import locale
locale.getpreferredencoding = lambda: "UTF-8"



In [60]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = 'bangla_gec_model'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def correct_grammar(input_text,num_return_sequences,input_len):
  batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=input_len, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=input_len,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text


In [61]:
references,predictions = [],[]
test_d = test_df[test_df['Error']==1]
test_d_sentence = test_d['Comment'].tolist()
test_d_len = test_d['input_token_len'].tolist()
test_d_ground = test_d['Correct Form'].tolist()
import nltk
nltk.download('punkt')
from nltk.util import ngrams
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [62]:
def calculate(reference,hypothesis):
    # Tokenize sentences into words
    reference_tokens = nltk.word_tokenize(reference)
    hypothesis_tokens = nltk.word_tokenize(hypothesis)

    # Create n-grams for reference and hypothesis
    reference_1grams = list(ngrams(reference_tokens, 1))
    hypothesis_1grams = list(ngrams(hypothesis_tokens, 1))
    reference_2grams = list(ngrams(reference_tokens, 2))
    hypothesis_2grams = list(ngrams(hypothesis_tokens, 2))

    # Calculate ROUGE scores
    rouge1_precision = len(set(reference_1grams).intersection(hypothesis_1grams)) / len(reference_1grams)
    rouge1_recall = len(set(reference_1grams).intersection(hypothesis_1grams)) / len(hypothesis_1grams)
    rouge2_precision = len(set(reference_2grams).intersection(hypothesis_2grams)) / len(reference_2grams)
    rouge2_recall = len(set(reference_2grams).intersection(hypothesis_2grams)) / len(hypothesis_2grams)

    # Calculate ROUGE-L using NLTK's sentence_bleu function
    smooth = SmoothingFunction().method4
    rougeL = sentence_bleu([reference_tokens], hypothesis_tokens, smoothing_function=smooth)
    d = {
        "rouge1_precision":rouge1_precision,
        "rouge1_recall":rouge1_recall,
        "rouge2_precision":rouge2_precision,
        "rouge2_recall":rouge2_recall,
        "rouge_l":rougeL
    }
    return d

In [63]:
import nltk
from nltk.util import ngrams

# Function to calculate ROUGE-1, ROUGE-2, and ROUGE-L scores for a pair of texts
def calculate_rouge_scores(reference_tokens, system_tokens):
    def lcs(X, Y):
        m, n = len(X), len(Y)
        dp = [[0] * (n + 1) for _ in range(m + 1)]

        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if X[i - 1] == Y[j - 1]:
                    dp[i][j] = dp[i - 1][j - 1] + 1
                else:
                    dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])

        return dp[m][n]

    # Calculate ROUGE-1 (unigram) scores
    reference_unigrams = set(reference_tokens)
    system_unigrams = set(system_tokens)
    overlap_rouge1 = len(reference_unigrams.intersection(system_unigrams))
    precision_rouge1 = overlap_rouge1 / len(system_unigrams)
    recall_rouge1 = overlap_rouge1 / len(reference_unigrams)
    r1_t = 1 if precision_rouge1 + recall_rouge1 == 0 else 0
    f1_rouge1 = 2 * (precision_rouge1 * recall_rouge1) / (precision_rouge1 + recall_rouge1 + r1_t)

    # Calculate ROUGE-2 (bigram) scores
    reference_bigrams = set(ngrams(reference_tokens, 2))
    system_bigrams = set(ngrams(system_tokens, 2))
    overlap_rouge2 = len(reference_bigrams.intersection(system_bigrams))
    precision_rouge2 = overlap_rouge2 / len(system_bigrams)
    recall_rouge2 = overlap_rouge2 / len(reference_bigrams)
    r2_t = 1 if precision_rouge2 + recall_rouge2 == 0 else 1
    f1_rouge2 = 2 * (precision_rouge2 * recall_rouge2) / (precision_rouge2 + recall_rouge2 + r2_t)

    # Calculate ROUGE-L scores
    lcs_length = lcs(reference_tokens, system_tokens)
    precision_rougeL = lcs_length / len(system_tokens)
    recall_rougeL = lcs_length / len(reference_tokens)
    rL_t = 1 if precision_rougeL + recall_rougeL == 0 else 0
    f1_rougeL = 2 * (precision_rougeL * recall_rougeL) / (precision_rougeL + recall_rougeL + rL_t)

    return {
        'ROUGE-1 Precision': precision_rouge1,
        'ROUGE-1 Recall': recall_rouge1,
        'ROUGE-1 F1': f1_rouge1,
        'ROUGE-2 Precision': precision_rouge2,
        'ROUGE-2 Recall': recall_rouge2,
        'ROUGE-2 F1': f1_rouge2,
        'ROUGE-L Precision': precision_rougeL,
        'ROUGE-L Recall': recall_rougeL,
        'ROUGE-L F1': f1_rougeL,
    }

# Function to calculate the average of ROUGE scores for an array of text pairs
def calculate_average_rouge_scores(reference_texts, system_texts):
    total_scores = {
        'ROUGE-1 Precision': 0,
        'ROUGE-1 Recall': 0,
        'ROUGE-1 F1': 0,
        'ROUGE-2 Precision': 0,
        'ROUGE-2 Recall': 0,
        'ROUGE-2 F1': 0,
        'ROUGE-L Precision': 0,
        'ROUGE-L Recall': 0,
        'ROUGE-L F1': 0,
    }

    num_pairs = len(reference_texts)

    for i in range(num_pairs):
        reference_text = reference_texts[i]
        system_text = system_texts[i]

        reference_tokens = nltk.word_tokenize(reference_text)
        system_tokens = nltk.word_tokenize(system_text)

        scores = calculate_rouge_scores(reference_tokens, system_tokens)

        for key, value in scores.items():
            total_scores[key] += value

    # Calculate the average scores
    average_scores = {key: value / num_pairs for key, value in total_scores.items()}
    
    return average_scores

# Example usage with an array of reference and system texts
reference_texts = test_d_ground
system_texts = [correct_grammar(test_d_sentence[i],num_return_sequences=2,input_len=test_d_len[i])[0] for i in range(len(test_d_sentence)) ]

average_scores = calculate_average_rouge_scores(reference_texts, system_texts)
print("Average ROUGE Scores:")
for key, value in average_scores.items():
    print(key + ": {:.4f}".format(value))

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Average ROUGE Scores:
ROUGE-1 Precision: 0.8357
ROUGE-1 Recall: 0.8343
ROUGE-1 F1: 0.8343
ROUGE-2 Precision: 0.7073
ROUGE-2 Recall: 0.7052
ROUGE-2 F1: 0.4246
ROUGE-L Precision: 0.8361
ROUGE-L Recall: 0.8342
ROUGE-L F1: 0.8344


In [64]:
# random data

r = train_df['Comment'].tolist()[:20] + test_df['Comment'].tolist()[:20]
r_ln = train_df['input_token_len'].tolist()[:20] + test_df['input_token_len'].tolist()[:20]
reference_texts = r
for i in range(len(r)):
    print(f'[input:   ] {r[i]} and [output:   ] {correct_grammar(r[i],num_return_sequences=2,input_len=r_ln[i])[0]}')


[input:   ] কাদের কি খেলব কাদের তো খেলতেই পারে না and [output:   ] কাদের কি খেলব কাদের তো খেলতেই পারে না
[input:   ] এসব করে আরো কোন ঠাসা হবে and [output:   ] এসব করে আরো কোন ঠাসা হবে
[input:   ] যুগ যুগ ধরে আমাদের মনে গেথে থাকবে এ গান and [output:   ] যুগ যুগ ধরে আমাদের মনে থাকবে এ গান
[input:   ] আচছা আপু এলাজী থাকলে টিকা নেওয়া জাবেনা and [output:   ] আচছা আপু এলাজী থাকলে টিকা নেওয়া যাবেনা
[input:   ] হে আল্লাহ এই জালিমদের থেকে আমাদের সন্তান সন্তদের কে আপনি হেফাজত করেন and [output:   ] হে আল্লাহ এই জালিমদের থেকে আমাদের সন্তান সন্তদের কে আপনি হেফাজত করেন
[input:   ] আমার মতে মাহমুদুল্লাহ রিয়াদ বাংলাদেশের সেরা খেলোয়াড় and [output:   ] আমার মতে মাহমুদুল্লাহ রিয়াদ বাংলাদেশের সেরা খেলোয়াড়
[input:   ] কথা গুলো শুনে কান্না করে দিয়েছি কারণ আমি ও এতীম এই পৃথিবীতে এতীম কোন দাম নেই পৃথিবীর মানুষ গুলো বড়ো স্বার্থ পর and [output:   ] কথা গুলো শুনে কান্না করে দিয়েছি কারণ আমি ও এতীম এই পৃথিবীতে এতীম কোন দাম নেই পৃথিবীর মানুষ গুলো বড়ো স্বার্থ পর
[input:   ] ইয়া আল্লাহ্ তুমিই রক্ষার মালি

In [66]:
# error data

import time

r = (test_df[test_df['Error']==1])['Comment'].tolist()
r_ln = (test_df[test_df['Error']==1])['input_token_len'].tolist()
reference_texts = r
start_time = time.time()
for i in range(len(r)):
    x = r[i]
    y = correct_grammar(r[i],num_return_sequences=2,input_len=r_ln[i])[0]
#     print(f'[input:   ] {r[i]} and [output:   ] {correct_grammar(r[i],num_return_sequences=2,input_len=r_ln[i])[0]}')
end_time = time.time()
num_iterations = len(r)
average_inference_time = (end_time - start_time) / num_iterations
print(f"Total Inference Time: {end_time-start_time}")
print(f"Average Inference Time: {average_inference_time:.4f} seconds")

Total Inference Time: 108.0009708404541
Average Inference Time: 0.1281 seconds


In [67]:
r_gn = (test_df[test_df['Error']==1])['Correct Form'].tolist()

In [70]:
predicted_score,predicted_sentences = [],[]
for i in range(len(r_gn)):
    x = r_gn[i]
    y = correct_grammar(r[i],num_return_sequences=2,input_len=r_ln[i])[0]
    predicted_sentences.append(y)
    rouge_score_v = calculate_rouge_scores(x, y)
    predicted_score.append(rouge_score_v['ROUGE-L F1'])
# test_df['ROUGE-L F1'] = predicted_sentence



In [72]:
e_df = test_df[test_df['Error']==1]
e_df['ROUGE-L F1'] = predicted_score
e_df['Predicted Form'] = predicted_sentences


/tmp/ipykernel_28/2934693448.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['ROUGE-L F1'] = predicted_score
/tmp/ipykernel_28/2934693448.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['Predicted Form'] = predicted_sentences


In [73]:
e_df.sort_values(by='ROUGE-L F1', inplace=True,ascending=False)

/tmp/ipykernel_28/3169983084.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df.sort_values(by='ROUGE-L F1', inplace=True,ascending=False)


In [75]:
e_df.head(20)[1:]

,Video Title,Genre,Comment,Error,Category,Correct Form,input_token_len,ROUGE-L F1,Predicted Form
858,শেখ মুজিব হত্যার পর জেনারেল জিয়া যে মন্তব্য করেছিলেন,Politics,আমাকে এজমা আছে আমি কি টিকা দিতে পারবো,1,Grammatical,আমার এজমা আছে আমি কি টিকা দিতে পারবো,10,1.0,আমার এজমা আছে আমি কি টিকা দিতে পারবো
874,বাধ্য হয়েই সুযোগ দেয়া হয়েছে তামিমকে ? | Tamim Iqbal | Riyad | Cricket | Ekattor TV,Sports,ধন্যবাদ ভাই রিসাদ আজিম ভাই সত্য টা তুলে দরলেন,1,Spelling,ধন্যবাদ ভাই রিসাদ আজিম ভাই সত্য টা তুলে ধরলেন,12,1.0,ধন্যবাদ ভাই রিসাদ আজিম ভাই সত্য টা তুলে ধরলেন
872,তুরস্কের চেয়ে ভয়াবহ ভূমিকম্পের ঝুঁকিতে বাংলাদেশ | BBC Bangla,News,স্টার জলসা আর জি বাংলা নষ্ট করলো আমাকে সোনার বাংলা,1,Grammatical,স্টার জলসা আর জি বাংলা নষ্ট করলো আমার সোনার বাংলা,11,1.0,স্টার জলসা আর জি বাংলা নষ্ট করলো আমার সোনার বাংলা
871,আলু দিয়ে মুরগির ঝোল বানানোর সেরা পদ্ধতি| New style chicken curry recipe in bengali |Atanur Rannaghar,Miscellaneous,দাদা সরিষা তেল না দিয়ে সয়াবিন তেল ব্যবহার করা যাবা,1,Grammatical,দাদা সরিষা তেল না দিয়ে সয়াবিন তেল ব্যবহার করা যাবে,12,1.0,দাদা সরিষা তেল না দিয়ে সয়াবিন তেল ব্যবহার করা যাবে
869,আইপিএলের নিলাম তালিকায় পাঁচ বাংলাদেশি | IPL | Bangladeshi | Khelajog | Ekattor TV,Sports,আর কেউ না খেললেও সাকিব খেলবে তাতে সন্দেহো নেই,1,Spelling,আর কেউ না খেললেও সাকিব খেলবে তাতে সন্দেহ নেই,12,1.0,আর কেউ না খেললেও সাকিব খেলবে তাতে সন্দেহ নেই
309,Turkey earthquake: সিরিয়ার ধ্বংসস্তূপ থেকে সুস্থ নবজাতক উদ্ধার,News,ইনশাআল্লাহ আমাদের দেশ তুস্কের মতো হবে না ইনশাআল্লাহ লাখো লাখো আলেমের দোয়া এবং চোখের পানি তাহাজ্জুদ নামাজের মতো গুরুত্বপূর্ণ আমল আছে অনেক আল্লাহ ওলার দোয়া আছে আমাদের দেশের মানুসের উপর আমরা আল্লাহ উপর নির্ভর করে থাকি আসুন সবাই মিলে আল্লাহর কাছে নামাজ পড়ে দোয়া করি এবং ভালো হয়ে যাই,1,Spelling,ইনশাআল্লাহ আমাদের দেশ তুস্কের মতো হবে না ইনশাআল্লাহ লাখো লাখো আলেমের দোয়া এবং চোখের পানি তাহাজ্জুদ নামাজের মতো গুরুত্বপূর্ণ আমল আছে অনেক আল্লাহ ওলার দোয়া আছে আমাদের দেশের মানুষের উপর আমরা আল্লাহ উপর নির্ভর করে থাকি আসুন সবাই মিলে আল্লাহর কাছে নামাজ পড়ে দোয়া করি এবং ভালো হয়ে যাই,57,1.0,ইনশাআল্লাহ আমাদের দেশ তুস্কের মতো হবে না ইনশাআল্লাহ লাখো লাখো আলেমের দোয়া এবং চোখের পানি তাহাজ্জুদ নামাজের মতো গুরুত্বপূর্ণ আমল আছে অনেক আল্লাহ ওলার দোয়া আছে আমাদের দেশের মানুষের উপর আমরা আল্লাহ উপর নির্ভর করে থাকি আসুন সবাই মিলে আল্লাহর কাছে নামাজ পড়ে দোয়া করি এবং ভালো হয়ে যাই
1542,বাহুবলী সিনেমার শ্যুটিং হয়েছিলো কেরালার যে জলপ্রপাতে || Bahubali Waterfalls | Athirapilly Waterfalls,Entertainment,সুমন ভাই আমাদের প্রিয় একটি মানুস তার ব্লগ দেখে আমাদের প্রাণটা জুড়িয়ে যায়,1,Spelling,সুমন ভাই আমাদের প্রিয় একটি মানুষ তার ব্লগ দেখে আমাদের প্রাণটা জুড়িয়ে যায়,16,1.0,সুমন ভাই আমাদের প্রিয় একটি মানুষ তার ব্লগ দেখে আমাদের প্রাণটা জুড়িয়ে যায়
1841,Oporadhi | Ankur Mahamud Feat Arman Alif | Bangla Song 2018 | Official Video,Entertainment,আমাকে মত কে কে আছেন এখনো সময় পেলে গানটা শুনেন,1,Grammatical,আমার মত কে কে আছেন এখনো সময় পেলে গানটা শুনেন,12,1.0,আমার মত কে কে আছেন এখনো সময় পেলে গানটা শুনেন
312,করোনা ভাইরাস ভ্যাকসিন: টিকা নিতে আগ্রহী হলে যেসব বিষয় আপনার জানা থাকা জরুরি,News,তালেবান কে স্যালুট জানাই সারাবিশ্বে আমাকে মত তারাই হকের পথে,1,Grammatical,তালেবান কে স্যালুট জানাই সারাবিশ্বে আমার মত তারাই হকের পথে,11,1.0,তালেবান কে স্যালুট জানাই সারাবিশ্বে আমার মত তারাই হকের পথে
860,তুরস্কের চেয়ে ভয়াবহ ভূমিকম্পের ঝুঁকিতে বাংলাদেশ | BBC Bangla,News,ভাইয়ের জন্য সবাই দোয়া করবােন ভাইয়ের জন্য সবাই দোয়া করবেন,1,Grammatical,ভাইয়ের জন্য সবাই দোয়া করবেন ভাইয়ের জন্য সবাই দোয়া করবেন,12,1.0,ভাইয়ের জন্য সবাই দোয়া করবেন ভাইয়ের জন্য সবাই দোয়া করবেন


In [76]:
e_df.tail(20)

,Video Title,Genre,Comment,Error,Category,Correct Form,input_token_len,ROUGE-L F1,Predicted Form
313,বিদ্যুৎ কেন নাই ব্যাখ্যা দিলেন প্রধানমন্ত্রী | Load Shedding Bangladesh | Sheikh Hasina | Somoy TV,News,কয়লা আছে ঠিকই মাগার ট্যেকা নাই,1,Spelling,কয়লা আছে ঠিকই কিন্তু টাকা নাই,10,0.754098,কয়লা আছে ঠিকই মাগার ট্যেকা নাই
354,ইতিহাসের সবথেকে ভয়ংকর দুর্গ | যেখানে একবার ঢুকলে কেউ আর ফেরত আসতো না | History of Daulatabad Fort,Miscellaneous,এই কেল্লা সত্যিই সিভিল ইঞ্জিনিয়ারিং এর আশ্চর্য ইতিহাস,1,Code Switching,এই কেল্লা সত্যিই নির্মাণ প্রকৌশলী এর আশ্চর্য ইতিহাস,9,0.742857,এই কেল্লা সত্যিই সিভিল ইঞ্জিনিয়ারিং এর আশ্চর্য ইতিহাস
1367,আমি বিশ্বের সবচেয়ে উষ্ণ স্থানে গিয়েছিলাম (৭০.৭°সে.) লুত মরুভূমি,Miscellaneous,অসাধারণ একটি ডকুমেন্টারি চ্যানেল,1,Code Switching,অসাধারণ একটি তথ্যচিত্র চ্যানেল,5,0.741935,অসাধারণ একটি ডকুমেন্টারি চ্যানেল
1493,Xiaomi 13 Lite - এমন ফোনই আমরা চাই !,Miscellaneous,শাওমির ফোন আমারে ফ্রী দিলেও তো আমি ইউজ করবো থার্ড ক্লাস মার্কা,1,Code Switching,শাওমির ফোন আমাকে বিনামূল্যে দিলেও তো আমি ব্যবহার করবো তৃতীয় ক্লাস মার্কা,16,0.740741,শাওমির ফোন আমারে ফ্রী দিলেও তো আমি ইউজ করবো থার্ড ক্লাস মার্কা
335,পশুদের মজার কর্মকান্ড ক্যামেরায় ধরা পড়া | Funny Animals Video 2022 (Part-3) | mayajaal,Entertainment,ডায়লোগ গুলো বেশি বেশি হইছে,1,Spelling,সংলাপ গুলো বেশি বেশি হয়েছে,8,0.740741,ডায়লোগ গুলো বেশি বেশি হইছে
1403,SURONGO | Official Foretaste | Afran Nisho | Tama Mirza | Raihan Rafi | Alpha-i | Chorki,Entertainment,এইটা কোন মুভির ট্রেইলার রইল একটা নাটক,1,Code Switching,এইটা কোন চলচ্চিত্রের খন্ড রইল একটা নাটক,9,0.736842,এইটা কোন মুভির ট্রেইলার রইল একটা নাটক
1595,Nothing Phone 1 First Time Unboxing & Impression 🇧🇩 | ATC,Miscellaneous,মিড বাজেটে ওয়াটারপ্রুফ ফোন কোনটা ভালো হবে,1,Code Switching,মধ্য বাজেটে জলরোধী ফোন কোনটা ভালো হবে,11,0.734177,মিড বাজেটে ওয়াটারপ্রুফ ফোন কোনটা ভালো হবে
394,ঋতু পরিবর্তন এবং মরুভূমি Season change on earth and Desert explained in Bangla Ep 107,Miscellaneous,প্রথম ভিও আমার,1,Code Switching,প্রথম দর্শন আমার,5,0.733333,প্রথম ভিও আমার
1675,[২য় পর্ব] সেরা কয়েকটি দৃষ্টিভ্রম | Top optical and sound illusion bangla|,Miscellaneous,এমন ভিডিও আরো চাই প্লিজ,1,Code Switching,এমন ভিডিও আরো চাই অনুগ্রহ করে,6,0.730769,এমন ভিডিও আরো চাই প্লিজ
342,ছায়াপথ বা গ্যালাক্সি | কি কেন কিভাবে | Galaxy | Ki Keno Kivabe,Miscellaneous,ভাইয়া ব্লকহোল নিয়ে একটি ভিডিও বানাবেন প্লিজ,1,Code Switching,ভাইয়া কৃষ্ণ গহ্বর নিয়ে একটি ভিডিও বানাবেন অনুগ্রহ করে,10,0.720000,ভাইয়া ব্লকহোল নিয়ে একটি ভিডিও বানাবেন প্লিজ
